# Linear Approximation

Bu Notebook, Cartpole probleminin lineer fonksiyon yakinsama ile cozumunu icermektedir.

In [ ]:
# Import Stuff
import gym
from IPython import display
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

%matplotlib inline

CSS = """
.output {
    align-items: center;
}
"""

display.HTML('<style>{}</style>'.format(CSS))

def show_state(env, e, t):
    plt.figure(3, figsize=(10, 10))
    plt.clf()
    plt.imshow(env.render(mode='rgb_array'))
    plt.title("Episode: {}, Step: {}".format(e, t))
    plt.axis('off')

    display.clear_output(wait=True)
    display.display(plt.gcf())

n_episodes = 500
gamma = 0.99
epsilon = 0.2
alpha = 0.1

# Cartpole

Bir cubuk hareket edebilen bir araca serbest bir eklem ile baglidir. Bu arac, surtunmesiz bir ortamda saga sola hareket etmektedir ve yarattigi ivme cubugu ters yonde hareket ettirmektedir. Amacimiz, araci hareket saga ya da sola hareket ettirerek cubugun dusmesini engellemektir. Cubugun devrilmedigi her anda +1 odulu elde edilir eger cubuk 15 dereceden fazla aci yaparsa veya arac merkezden 2.4 unite uzaklasirsa episode biter.

Bu problemde stateler sureklidir (continuous). Bu nedenle tablo olusturmak yerine stateleri Q degerlerine yansitacak parametreleri ogrenmemiz gerekmekte. Bu ornekte lineer bir sekilde bu fonksiyonu parametrize edecegiz.

$$
Q_W(s, .) = Ws
$$

Her iterasyon icin, policemizden bir aksiyon sececegiz ($\epsilon$-greedy). Secilen aksiyona gore suan bulundugumuz satetin Q degerini gelecekte elde edilecek odulleri makzimize edecek sekilde guncelleyecegiz. Once TD hatasini (Temporal difference error) hesaplamamiz gerekmekte.

$$
\delta = R + \gamma Q_W(S', A') - Q_W(S, A)
$$

Bu hatayi minimize etmek istiyoruz. $W$ parametrelerini $-\delta$'yi minimize edecek sekilde guncellemek icin, $Q$ degerlerinin secilen aksiyon icin  degisimi yonunde ilerlememiz gerekmektedir, bu bilgiyi de bize gradyan vermektedir.

$$
W \leftarrow W + \alpha\delta\nabla_WQ(S,A)
$$

In [ ]:
# Fonksiyon parametrelerini olusturalim
# State buyuklugu 4, aksiyon sayisi 2
W = np.zeros((2, 4))

# Q Fonksiyonu
def f(W, s):
    return np.matmul(W, s)

# Verilen bir aksiyonun Q degeri
def q(W, s, a):
    return f(W, s)[a]

# Q degerinin parametrelere gore gradyani
def dq(W, s, a):
    d = np.zeros_like(W)
    d[a] = s
    return d

# epsilon-greedy policesi
def policy(W, s, eps=None):
    if eps:
        nA = W.shape[0]
        if np.random.rand() < eps:
            return np.random.randint(nA)
    return np.argmax(f(W, s))

# Greedy police kullanan fonksiyon
def run_env(env, W):
    state = env.reset()
    done = False
    t = 0

    while not done:
        t += 1
        show_state(env, 1, t)

        action = policy(W, state)
        next_state, reward, done, info = env.step(action)
        state = next_state

In [ ]:
# Egitilmemis
env = gym.make('CartPole-v0')
run_env(env, W)

In [ ]:
rewards = []
W = np.zeros((2, 4))

# Her bir episode icin
for e in tqdm(range(1, n_episodes+1)):
    state = env.reset()
    done = False
    t = 0
    total_reward = 0
    
    # Bitmedigi surece
    while not done:
        t += 1
        
        # Epsilon-greedy ile aksiyon sec
        action = policy(W, state, epsilon)
        # Aksiyon al
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        
        if done:
            # Eger bitmisse sonraki aksionu dahil etme
            W += alpha * (reward - q(W, state, action)) * dq(W, state, action)
        else:
            # Bitmemisse sonraki aksiyonu da dahil et
            next_action = policy(W, next_state, epsilon)
            diff = reward - gamma*q(W, next_state, next_action) - q(W, state, action)
        
            W += alpha * diff * dq(W, state, action)
            
        state = next_state
       
    rewards.append(total_reward)
    
    if e % 100 == 0:
        epsilon /= 2.
    
env.close()

In [ ]:
plt.plot(rewards)

In [ ]:
# Egitildikten sonra
run_env(env, W)

# Sonuc

Deadly triad